Exercicio 1: crie uma função de ML_pipeline

Nela tem como parametros: dados de treino, dados de teste, modelo, tipo de treinamento(mediante um parametro string) que diz 'kfold' , 'stratifiedkfold' , 'groupstratkfold', número de folhas, metodo de avaliação, qual score(loss function)

tem como objetivo algo importante como

Treinamento, fixando o tipo de treino: k fold, leave one out, group k fold, stratified group k fold


Ela retorna

model, resultado nos dados de treino em cada folha(lista), resultado nos dados de teste(lista, em cada folha tbm), valor médio nos dados de treino, valor médio nos dados de teste


In [2]:
import pandas as pd
import numpy as np
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
from tqdm import tqdm

In [3]:
train_test_pct_split = 0.3
data_set = pd.read_csv('./heart_failure_clinical_records.csv')
test = data_set.sample(frac = train_test_pct_split)
train = data_set.loc[~data_set.index.isin(test.index)]

### convensão coluna target esta sempre ao final do df

In [21]:
class ML_pipeline():
    """Nessa Classe vc pode contruir um objeto que tem as propriedades de treinar
    um modelo utilizando diversas tecnicas de cross-validation incluindo:
    -> k_fold \n
    -> stratified_kfold \n
    -> leave_one_out_kfold \n
    -> group_kfold \n
    -> groupstrat_kfold \n

    É esperado que a coluna target seja a ultima coluna do dataframe de treino
    """
    def __init__(self,n,data):
        self.n = n
        self.data = data

    def k_fold(self,model,score):
        """Essa função gera os k fold treinamentos para modelo.
        Init: O df é aleatoriamente dividido e é determinado a quantidade de amostras no dado de teste
        Para cada iteração as partes são dividas em treino e teste.
        Todo o dado é utilizado percorrendo em conjuntos de partes

        Args:
            n (int): numero de folds do k_fold
            data (pd.Dataframe): dataframe com coluna resposta ao final das de sua coluna
            model (sklearn_mdoel): modelo escolhido do pacote sklearn
            loss_function (sklearn_loss_function): score utilizado

        Returns:
            dict: dicionário com melhor modelo,
            resultado para os n folds (teste e treino) e média do score (teste e treino) 
        """
        df = self.data.sample(n=len(self.data)).copy()
        partes_iguais = math.ceil(len(df)/self.n) 
        folds = {'score_treino': [],
                'score_teste':[]}
        trained_model = model
        modelos = []
        for i in tqdm(range(1,self.n+1)):
            test = df.iloc[partes_iguais*(i-1):partes_iguais*i]
            train = df.loc[~df.index.isin(test.index)]

            X_train = train[df.columns[:-1]]
            y_train = train[df.columns[-1]]

            X_test = test[df.columns[:-1]]
            y_test = test[df.columns[-1]]

            
            trained_model = trained_model.fit(X_train,y_train)
            pred_train = trained_model.predict(X_train)
            pred_test = trained_model.predict(X_test)

            score_treino = score(y_train,pred_train)
            score_teste = score(y_test,pred_test)

            modelos.append(trained_model)

            folds['score_teste'].append(score_teste)
            folds['score_treino'].append(score_treino)
            folds['melhor_modelo'] = modelos[np.argmax(folds['score_teste'])]

            folds['media_treino'] = np.mean(folds['score_treino'])
            folds['media_teste'] = np.mean(folds['score_teste'])
        

        return folds
    
    def stratified_kfold(self,model,loss_function):
        """Essa função gera os kfold dados de treinamento mantendo a proporção entre as N classes para 
        cada agrupamento de treinamento e teste.

        Args:
            n (int): numero de folds
            data (pd.DataFrame): dataframe com coluna resposta ao final das de sua coluna
            model (sklearn_model): modelo escolhido do pacote sklearn
            loss_function (sklearn_loss_function): score utilizado

        Returns:
            dict: dicionário com melhor modelo,
            resultado para os n folds (teste e treino) e média do score (teste e treino) 
        """
        df = self.data.sample(n=len(self.data)).copy()
        folds = {'score_treino': [],
                'score_teste':[]}
        modelos = []
        grupos_de_classes = df.groupby(df.columns[-1])
        proporcao = grupos_de_classes.apply(len)/len(df)
        partes_iguais_proporcao = (len(df)/self.n * proporcao).apply(math.ceil)

        for i in tqdm(range(1,self.n+1)):
            classes = {'train':[],
                    'test':[]}
            for k,v in grupos_de_classes:
                test_class = v.iloc[partes_iguais_proporcao[k]*(i-1):partes_iguais_proporcao[k]*i] # garantia de manter proporcao em dados de teste
                train_class = v.loc[~v.index.isin(test_class.index)] # garantia de manter a proporcao em dados de treino
                classes['train'].append(train_class)
                classes['test'].append(test_class)

            train = pd.concat(classes['train'])
            test = pd.concat(classes['test'])

            X_train = train[df.columns[:-1]]
            y_train = train[df.columns[-1]]

            X_test = test[df.columns[:-1]]
            y_test = test[df.columns[-1]]

            
            model = model.fit(X_train,y_train)
            pred_train = model.predict(X_train)
            pred_test = model.predict(X_test)

            score_treino = loss_function(y_train,pred_train)
            score_teste = loss_function(y_test,pred_test)

            modelos.append(model)

            folds['score_teste'].append(score_teste)
            folds['score_treino'].append(score_treino)
            folds['melhor_modelo'] = modelos[np.argmax(folds['score_teste'])]

            folds['media_treino'] = np.mean(folds['score_treino'])
            folds['media_teste'] = np.mean(folds['score_teste'])

        return folds
    
    def leave_one_out_kfold(self,model,loss_function)->dict:
        """Essa função gera os k fold treinamentos para modelo.
        Init: O df é aleatoriamente dividido e é determinado a quantidade de amostras no dado de teste
        Para cada iteração as partes são dividas em treino e teste.
        Todo o dado é utilizado percorrendo em conjuntos de partes

        Args:
            n (int): numero de folds do k_fold
            data (pd.Dataframe): dataframe com coluna resposta ao final das de sua coluna
            model (sklearn_mdoel): modelo escolhido do pacote sklearn
            loss_function (sklearn_loss_function): score utilizado

        Returns:
            dict: dicionário com melhor modelo,
            resultado para os n folds (teste e treino) e média do score (teste e treino) 
        """
        df = self.data.sample(n=len(self.data)).copy()
        n = len(self.data)
        partes_iguais = math.ceil(len(df)/n)
        folds = {'score_treino': [],
                'score_teste':[]}
        modelos = []
        for i in tqdm(range(1,n+1)):
            test = df.iloc[partes_iguais*(i-1):partes_iguais*i]
            train = df.loc[~df.index.isin(test.index)]

            X_train = train[df.columns[:-1]]
            y_train = train[df.columns[-1]]

            X_test = test[df.columns[:-1]]
            y_test = test[df.columns[-1]]

            
            model = model.fit(X_train,y_train)
            pred_train = model.predict(X_train)
            pred_test = model.predict(X_test)

            score_treino = loss_function(y_train,pred_train)
            score_teste = loss_function(y_test,pred_test)

            modelos.append(model)

            folds['score_teste'].append(score_teste)
            folds['score_treino'].append(score_treino)
            folds['melhor_modelo'] = modelos[np.argmax(folds['score_teste'])]

            folds['media_treino'] = np.mean(folds['score_treino'])
            folds['media_teste'] = np.mean(folds['score_teste'])

        return folds
    
    def group_kfold(self,group_col:str,model,loss_function):
        """essa funcao crie n folds para cada grupo exclusivo. Isolando o treinamento apenas ao conjunto de dados complementar ao grupo

        Args:
            n (int): numero de folds por grupo
            group_col (str): coluna do grupo
            data (pd.DataFrame): dados de treino
            model (_type_): modelo
            loss_function (_type_): score

        Returns:
            dict: dicionario com informações do modelo
        """

        df = self.data.sample(n=len(self.data)).copy()
        folds = {'score_treino': [],
                'score_teste':[]}
        modelos = []
        grupos_de_classes = df.groupby(group_col)

        
        for _, v in tqdm(grupos_de_classes): # para cada grupo é feito o treinamento excluindo o grupo em n partes iguais
            test = v
            train = df.loc[~df.index.isin(test.index)] # sempre treinar os dados no conjunto de dados complementares ao grupo

            partes_iguais_test = math.ceil(len(test)/self.n)
            #partes_iguais_treino = math.ceil(len(train)/n)
            
            for i in range(1,self.n+1):
                test_fold = test.iloc[partes_iguais_test*(i-1):partes_iguais_test*i] # testar o modelo 
                #train_fold = train.iloc[partes_iguais_treino*(i-1):partes_iguais_treino*i] # talvez se quiser treinar o modelo em conjuntos do grupo complementar

                X_train = train[df.columns[:-1]]
                y_train = train[df.columns[-1]]
                X_test = test_fold[df.columns[:-1]]
                y_test = test_fold[df.columns[-1]]

                
                model = model.fit(X_train,y_train)
                pred_train = model.predict(X_train)
                pred_test = model.predict(X_test)

                score_treino = loss_function(y_train,pred_train)
                score_teste = loss_function(y_test,pred_test)

                modelos.append(model)

                folds['score_teste'].append(score_teste)
                folds['score_treino'].append(score_treino)

                folds['melhor_modelo'] = modelos[np.argmax(folds['score_teste'])]
                folds['media_treino'] = np.mean(folds['score_treino'])
                folds['media_teste'] = np.mean(folds['score_teste'])

        return folds
    
    def groupstrat_kfold(self,group_col:str,model,loss_function):
        """essa funcao crie n folds para cada grupo exclusivo. Isolando o treinamento apenas ao conjunto de dados complementar ao grupo respeitando
        a propocao de dados no conjunto de treino

        Args:
            n (int): numero de folds por grupo
            group_col (str): coluna do grupo
            data (pd.DataFrame): dados de treino
            model (_type_): modelo
            loss_function (_type_): score

        Returns:
            dict: dicionario com informações do modelo
        """

        df = self.data.sample(n=len(self.data)).copy()
        target_column = df.columns[-1]
        folds = {'score_treino': [],
                    'score_teste':[]}
        modelos = []
        grupos_de_classes = df.groupby(group_col)
        proporcao_target = df[target_column].value_counts()/len(df)

        for label_grupo, grupo in tqdm(grupos_de_classes): # para cada grupo é feito o treinamento excluindo o grupo em n partes iguais

            teste_ = grupo
            train_ = df.loc[~df.index.isin(grupo.index)]

            partes_iguais_test = ((len(teste_)/self.n)*proporcao_target).apply(math.ceil)
            partes_iguais_treino = ((len(train_)/self.n)*proporcao_target).apply(math.ceil)

            for i in range(1,self.n+1):
                classes = {'train':[],
                            'test':[]}
                
                for k in proporcao_target.index:
                    test_class = teste_.iloc[partes_iguais_test[k]*(i-1):partes_iguais_test[k]*i] # garantia de manter proporcao em dados de teste
                    train_class = train_.iloc[partes_iguais_treino[k]*(i-1):partes_iguais_treino[k]*i] # garantia de manter a proporcao em dados de treino


                    classes['train'].append(train_class)
                    classes['test'].append(test_class)


                test_fold = pd.concat(classes['test'])
                train_fold = pd.concat(classes['train'])


                X_train = train_fold[df.columns[:-1]]
                y_train = train_fold[target_column]

                X_test = test_fold[df.columns[:-1]]
                y_test = test_fold[target_column]

                
                model = model.fit(X_train,y_train)
                pred_train = model.predict(X_train)
                pred_test = model.predict(X_test)

                score_treino = loss_function(y_train,pred_train)
                score_teste = loss_function(y_test,pred_test)

                modelos.append(model)

                folds['score_teste'].append(score_teste)
                folds['score_treino'].append(score_treino)
                folds['melhor_modelo'] = modelos[np.argmax(folds['score_teste'])]

                folds['media_treino'] = np.mean(folds['score_treino'])
                folds['media_teste'] = np.mean(folds['score_teste'])

        return folds

    
   

In [22]:
data_pipe_line = ML_pipeline(2,train)

In [23]:
data_pipe_line.k_fold(RandomForestClassifier(),accuracy_score)

100%|██████████| 2/2 [00:00<00:00,  4.18it/s]


{'score_treino': [1.0, 0.9988571428571429],
 'score_teste': [0.9862857142857143, 0.9948571428571429],
 'melhor_modelo': RandomForestClassifier(),
 'media_treino': 0.9994285714285714,
 'media_teste': 0.9905714285714287}

In [24]:
data_pipe_line.stratified_kfold(RandomForestClassifier(),accuracy_score)

100%|██████████| 2/2 [00:00<00:00,  4.62it/s]


{'score_treino': [0.9988571428571429, 1.0],
 'score_teste': [0.9902857142857143, 0.992],
 'melhor_modelo': RandomForestClassifier(),
 'media_treino': 0.9994285714285714,
 'media_teste': 0.9911428571428571}

In [25]:
data_pipe_line.leave_one_out_kfold(RandomForestClassifier(),accuracy_score)

100%|██████████| 3500/3500 [16:48<00:00,  3.47it/s]


{'score_treino': [0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.9994284081166047,
  0.999428408116

In [26]:
data_pipe_line.group_kfold('sex',RandomForestClassifier(),accuracy_score)

100%|██████████| 2/2 [00:00<00:00,  2.63it/s]


{'score_treino': [0.9991181657848325, 0.9991181657848325, 1.0, 1.0],
 'score_teste': [0.8863636363636364,
  0.9253246753246753,
  0.8562610229276896,
  0.8597883597883598],
 'melhor_modelo': RandomForestClassifier(),
 'media_treino': 0.9995590828924162,
 'media_teste': 0.8819344236010902}

In [27]:
data_pipe_line.groupstrat_kfold('sex',RandomForestClassifier(),accuracy_score)

100%|██████████| 2/2 [00:00<00:00,  3.21it/s]


{'score_treino': [0.9982378854625551, 0.9991189427312775, 1.0, 1.0],
 'score_teste': [0.9189627228525121,
  0.9189627228525121,
  0.8387665198237886,
  0.8960352422907489],
 'melhor_modelo': RandomForestClassifier(),
 'media_treino': 0.9993392070484581,
 'media_teste': 0.8931818019548905}